In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from matplotlib.colors import ListedColormap
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# Load the dataset
iris = datasets.load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data[:, [0, 1]], iris.target, test_size=0.3, random_state=42)

def plot_knn_with_test_data(k, weight, p):
    model = KNeighborsClassifier(n_neighbors=k, weights=weight, p=p)
    model.fit(X_train, y_train)
    
    accuracy = model.score(X_test, y_test)
    
    # Create color maps
    cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
    cmap_bold = ['darkred', 'darkgreen', 'darkblue']
    
    x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
    y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
    
    xx, yy = np.meshgrid(np.arange(x_min, x_max, .01), np.arange(y_min, y_max, .01))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    
    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure(figsize=(12, 6))
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light, shading='auto')
    
    # Plot the training points
    for i, color in zip(range(3), cmap_bold):
        idx = np.where(y_train == i)
        plt.scatter(X_train[idx, 0], X_train[idx, 1], c=color, label=f"{iris.target_names[i]} (Train)",
                    edgecolor='black', s=20, alpha=0.6)
        
    predictions = model.predict(X_test)
    # Plot the testing points
    for i, color in zip(range(3), cmap_bold):
        idx = np.where(y_test == i)
        plt.scatter(X_test[idx, 0], X_test[idx, 1], c=color, label=f"{iris.target_names[i]} (Test)",
                    edgecolor='black', s=50, marker='D')

    # Highlight the misclassified points
    misclassified = X_test[y_test != predictions]
    plt.scatter(misclassified[:, 0], misclassified[:, 1], c='none', edgecolor='red', s=200, linewidth=1.5, marker='o')
    
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title(f"k-NN Classification Results (Test Accuracy: {accuracy:.2f})\n"
              f"K={k}, Weight={weight}, p={p}")
    plt.xlabel('Sepal length')
    plt.ylabel('Sepal width')
    plt.legend(loc='best')
    plt.show()

interactive_plot = interactive(
    plot_knn_with_test_data, 
    k=widgets.IntSlider(min=1, max=20, step=1, value=5), 
    weight=widgets.RadioButtons(options=['uniform', 'distance'], value='uniform'),
    p=widgets.FloatSlider(min=1, max=5, value=2)
)
output = interactive_plot.children[-1]
output.layout.height = '500px'
interactive_plot




interactive(children=(IntSlider(value=5, description='k', max=20, min=1), RadioButtons(description='weight', o…

### Understanding the Impact of K, Weight, and P in k-NN Algorithm

In the k-NN algorithm, three crucial parameters—`K`, `Weight`, and `P`—play a significant role in determining the model's performance and behavior. Let's delve deeper into the essence of each:

#### 1. **K Value:**
In k-NN, the `K` value represents the number of neighbors we consider to predict the class of a new, unseen instance.

- **Small K Values:**
  - **Pros:** Able to capture finer details of data.
  - **Cons:** More sensitive to noise, with a higher risk of overfitting.
- **Large K Values:**
  - **Pros:** Tends to make more stable and generalized predictions.
  - **Cons:** Potentially smoothens over the finer details of class boundaries, risking underfitting.

#### 2. **Weight:**
The `Weight` parameter determines the influence each of the `K` neighbors has on the final prediction.

- **Uniform Weight:**
  - Every neighbor has an equal vote, regardless of its distance from the test instance. It treats every neighbor with equal importance.
- **Distance Weight:**
  - Closer neighbors to the test instance have a greater influence on the prediction. The influence or the weight is inversely proportional to the distance of the neighbor from the test instance, allowing the model to be more sensitive to local variations.

#### 3. **P Value:**
The `P` parameter is associated with the Minkowski distance metric, a generalized distance metric that includes both Euclidean (p=2) and Manhattan (p=1) distances.

- **Small P Value (e.g., 1 - Manhattan Distance):**
  - Emphasizes the difference along individual dimensions and is effective in grid-like path structures.
- **Large P Value (e.g., 2 - Euclidean Distance):**
  - Considers the cumulative difference across dimensions, suitable for circular or elliptical path structures.

### Summary:
- **Adjusting K:** Manages the trade-off between noise sensitivity and detail smoothing, controlling the granularity of the classification.
- **Choosing Weight:** Decides the influence of neighbors, allowing either equal contribution or weighted contribution based on proximity.
- **Tuning P Value:** Affects the distance metric used, influencing the perception of 'closeness' between instances in the feature space.

Through interactive experimentation with these parameters, we can gain insights into their impact, refine the model's behavior, and optimize its performance for various scenarios.


## Curse of Dimensionalty

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

def plot_curse_of_dimensionality(num_points, max_dimension):
    def average_distance(num_dimensions):
        """Calculate the average pairwise Euclidean distance between random points in a given dimension."""
        # Generate random points
        points = np.random.rand(num_points, num_dimensions)

        total_distance = 0
        count = 0
        for i in range(num_points):
            for j in range(i+1, num_points):
                total_distance += np.linalg.norm(points[i] - points[j])
                count += 1

        return total_distance / count

    # Compute average distances for each dimension
    dimensions = range(1, max_dimension+1)
    distances = [average_distance(dim) for dim in dimensions]

    # Plot
    plt.figure(figsize=(10, 6))
    plt.plot(dimensions, distances)
    plt.xlabel('Dimensions')
    plt.ylabel('Average Distance')
    plt.title('Curse of Dimensionality: Average Distance Between Points')
    plt.grid(True)
    plt.show()

# Interactive widgets
points_slider = widgets.IntSlider(value=100, min=50, max=500, step=10, description="Num Points:")
dimension_slider = widgets.IntSlider(value=50, min=2, max=200, step=1, description="Max Dimension:")

interactive_plot = widgets.interactive(plot_curse_of_dimensionality, num_points=points_slider, max_dimension=dimension_slider)
output = interactive_plot.children[-1]
display(interactive_plot)


interactive(children=(IntSlider(value=100, description='Num Points:', max=500, min=50, step=10), IntSlider(val…

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

def plot_distance_distribution(num_points, num_dimensions, bins=30):
    """Visualize the distribution of distances in a given dimension."""
    # Generate random points
    points = np.random.rand(num_points, num_dimensions)
    
    distances = []
    for i in range(num_points):
        for j in range(i+1, num_points):
            distances.append(np.linalg.norm(points[i] - points[j])) ## L2 norm in default
            
    # Plot histogram of distances
    plt.hist(distances, bins=bins, alpha=0.7, label=f"{num_dimensions} dimensions")
    plt.legend()
    plt.xlabel('Distance')
    plt.ylabel('Number of Point Pairs')
    plt.title(f'Distribution of Pairwise Distances for {num_points} Points in {num_dimensions} Dimensions')
    plt.grid(True)
    plt.show()


# Interactive widgets
points_slider = widgets.IntSlider(value=100, min=50, max=500, step=10, description="Num Points:")
dimension_slider = widgets.IntSlider(value=10, min=2, max=2000, step=1, description="Dimension:")
bins_slider = widgets.IntSlider(value=30, min=10, max=100, step=1, description="Bins:")

interactive_plot = widgets.interactive(plot_distance_distribution, num_points=points_slider, num_dimensions=dimension_slider, bins=bins_slider)
display(interactive_plot)


interactive(children=(IntSlider(value=100, description='Num Points:', max=500, min=50, step=10), IntSlider(val…

### Understanding the Terminologies and Parameters in the Visualization:

#### 1. **Distance:**
   In the context of the visualized graph, "Distance" represents the computed spatial separation between pairs of data points in a high-dimensional space. It quantifies how far apart the individual data points are, using a defined metric like Euclidean or Manhattan, considering the specified number of dimensions (`max_dims`).

#### 2. **Bins:**
   "Bins" in the histogram refer to the intervals or range of values used to group the distances. Each bin represents a specific range of distances, and the frequency within each bin indicates how many distances in the dataset fall within that range. Adjusting the number of bins can provide a more refined or generalized view of the distance distributions, allowing for detailed analysis or broader overviews.

#### 3. **Interpretation of num_points and max_dims:**
   - **`num_points`:** Represents the number of data points generated in the high-dimensional space. Increasing this value will result in more distances calculated, offering a denser representation of the distance distribution in the specified dimensional space.
   - **`max_dims`:** Denotes the number of dimensions considered in the space where data points are generated. Increasing the number of dimensions (max_dims) will generally result in an increase in the average distance between data points, illustrating the concept of the "Curse of Dimensionality." Observing the variation in distance distribution with changing dimensions provides insights into how data behaves in high-dimensional spaces.

### Final Thoughts:
By interacting with and adjusting the parameters like `num_points`, `max_dims`, and `bins`, viewers can observe the nuances in distance distributions and frequency occurrences, gaining a practical understanding of the implications of high-dimensional spaces and the "Curse of Dimensionality" in distance-based algorithms like k-NN.


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

def curse_of_dimensionality(N=1000, K=10, dimensions=2):
    # Generate N random points in the unit hypercube
    points = np.random.rand(N, dimensions)
    
    # Choose a random point within the hypercube
    random_point = np.random.rand(dimensions)
    
    # Calculate the distances from the random point to all other points
    distances = np.linalg.norm(points - random_point, axis=1)
    
    # Get the indices of the K nearest points
    k_nearest_indices = np.argsort(distances)[:K]
    k_nearest_points = points[k_nearest_indices]
    
    # Calculate the bounding box (hypercube) for the K nearest points
    min_bounds = k_nearest_points.min(axis=0)
    max_bounds = k_nearest_points.max(axis=0)
    l = max(max_bounds - random_point)

    print(f"For {dimensions} dimensions, the side length 'l' of the cube required to encapsulate {K} nearest points is approximately {l:.4f}.")

    # Visualization for 2D case
    if dimensions == 2:
        fig, ax = plt.subplots()
        ax.scatter(points[:, 0], points[:, 1], color='blue', s=10, label="Random Points")
        ax.scatter(random_point[0], random_point[1], color='red', s=40, label="Randomly Chosen Point")
        
        # Draw a rectangle for the bounding box of K nearest points
        ax.add_patch(plt.Rectangle((random_point[0] - l/2, random_point[1] - l/2), l, l, fill=False, edgecolor='green', label=f"Side length {l:.4f}"))
        
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.set_aspect('equal', 'box')
        ax.legend()
        plt.show()

# Interactive widget
widgets.interactive(curse_of_dimensionality, N=(10, 1000, 10), K=(1, 100, 1), dimensions=(2, 100, 1))


interactive(children=(IntSlider(value=1000, description='N', max=1000, min=10, step=10), IntSlider(value=10, d…